In [1]:
import pandas as pd
import os

# 1. Configuração do Arquivo
# Verifica se o arquivo é 'raw_data.csv' ou apenas 'raw_data'
if os.path.exists('./data/raw_data.csv'):
    FILE_PATH = './data/raw_data.csv'
elif os.path.exists('./data/raw_data'):
    FILE_PATH = './data/raw_data'
else:
    raise FileNotFoundError("Arquivo não encontrado em ./data/. Verifique o nome.")

# 2. Definição de Colunas (Schema)
# Se o Verra mudou os nomes, o script vai avisar no erro.
COLS_MAP = {
    'Project Name': 'project_name',
    'Country/Area': 'country',
    'Project Type': 'project_type', 
    'Vintage': 'vintage',
    'Quantity Issued': 'quantity',
    'Retirement/Cancellation Date': 'retirement_date'
}

try:
    print(f"Lendo arquivo: {FILE_PATH} ...")
    
    # Ingestão
    df = pd.read_csv(
        FILE_PATH, 
        usecols=COLS_MAP.keys(),
        encoding='latin1', 
        low_memory=False
    )
    
    # Renomeação
    df = df.rename(columns=COLS_MAP)
    
    # 3. Limpeza de Tipos (Type Casting) - CRÍTICO
    # Removemos linhas vazias onde 'project_name' é nulo (lixo de formatação)
    df = df.dropna(subset=['project_name'])
    
    # Forçamos 'quantity' para numérico (remove potenciais vírgulas de milhar)
    # Ex: '1,000' vira 1000.0
    if df['quantity'].dtype == 'O': # Se for Object (string)
        df['quantity'] = df['quantity'].astype(str).str.replace(',', '').astype(float)
        
    # Convertemos Data (Strings para Datetime)
    df['retirement_date'] = pd.to_datetime(df['retirement_date'], errors='coerce')
    
    # Convertemos Vintage (Safra) para Inteiro
    df['vintage'] = pd.to_numeric(df['vintage'], errors='coerce').fillna(0).astype(int)

    print("-" * 30)
    print("DATASET AUDITADO E PRONTO")
    print("-" * 30)
    df.info() # Mostra o uso de memória e tipos reais
    display(df.head())

except ValueError as e:
    print(f"[ERRO DE SCHEMA] Coluna não encontrada. As colunas no arquivo são:")
    print(pd.read_csv(FILE_PATH, nrows=1).columns.tolist())

Lendo arquivo: ./data/raw_data.csv ...
[ERRO DE SCHEMA] Coluna não encontrada. As colunas no arquivo são:
['ID', 'Name', 'Proponent', 'Project Type', 'AFOLU Activities', 'Methodology', 'Status', 'Country/Area', 'Estimated Annual Emission Reductions', 'Region', 'Project Registration Date', 'Crediting Period Start Date', 'Crediting Period End Date']


In [2]:
import pandas as pd
import os

# 1. Configuração do Arquivo
# Verifica se o arquivo é 'raw_data.csv' ou apenas 'raw_data'
if os.path.exists('./data/raw_data.csv'):
    FILE_PATH = './data/raw_data.csv'
elif os.path.exists('./data/raw_data'):
    FILE_PATH = './data/raw_data'
else:
    raise FileNotFoundError("Arquivo não encontrado em ./data/. Verifique o nome.")

# 2. Definição de Colunas (Schema)
# Se o Verra mudou os nomes, o script vai avisar no erro.
COLS_MAP = {
    'Project Name': 'project_name',
    'Country/Area': 'country',
    'Project Type': 'project_type', 
    'Vintage': 'vintage',
    'Quantity Issued': 'quantity',
    'Retirement/Cancellation Date': 'retirement_date'
}

try:
    print(f"Lendo arquivo: {FILE_PATH} ...")
    
    # Ingestão
    df = pd.read_csv(
        FILE_PATH, 
        usecols=COLS_MAP.keys(),
        encoding='latin1', 
        low_memory=False
    )
    
    # Renomeação
    df = df.rename(columns=COLS_MAP)
    
    # 3. Limpeza de Tipos (Type Casting) - CRÍTICO
    # Removemos linhas vazias onde 'project_name' é nulo (lixo de formatação)
    df = df.dropna(subset=['project_name'])
    
    # Forçamos 'quantity' para numérico (remove potenciais vírgulas de milhar)
    # Ex: '1,000' vira 1000.0
    if df['quantity'].dtype == 'O': # Se for Object (string)
        df['quantity'] = df['quantity'].astype(str).str.replace(',', '').astype(float)
        
    # Convertemos Data (Strings para Datetime)
    df['retirement_date'] = pd.to_datetime(df['retirement_date'], errors='coerce')
    
    # Convertemos Vintage (Safra) para Inteiro
    df['vintage'] = pd.to_numeric(df['vintage'], errors='coerce').fillna(0).astype(int)

    print("-" * 30)
    print("DATASET AUDITADO E PRONTO")
    print("-" * 30)
    df.info() # Mostra o uso de memória e tipos reais
    display(df.head())

except ValueError as e:
    print(f"[ERRO DE SCHEMA] Coluna não encontrada. As colunas no arquivo são:")
    print(pd.read_csv(FILE_PATH, nrows=1).columns.tolist())

Lendo arquivo: ./data/raw_data.csv ...
[ERRO DE SCHEMA] Coluna não encontrada. As colunas no arquivo são:
['Issuance Date', 'Sustainable Development Goals', 'Vintage Start', 'Vintage End', 'ID', 'Name', 'Country/Area', 'Project Type', 'Methodology', 'Total Vintage Quantity', 'Quantity Issued', 'Serial Number', 'Additional Certifications', 'Retirement/Cancellation Date', 'Retirement Beneficiary', 'Retirement Reason', 'Retirement Details']


In [3]:
import pandas as pd
import os

# 1. Configuração do Arquivo
FILE_PATH = './data/raw_data.csv'

# 2. Definição de Colunas (Mapeamento Exato do Log)
COLS_MAP = {
    'Name': 'project_name',
    'Country/Area': 'country',
    'Project Type': 'project_type',
    'Vintage End': 'vintage',           # Usaremos o fim da safra como ano de referência
    'Quantity Issued': 'quantity',
    'Retirement/Cancellation Date': 'retirement_date'
}

if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"Arquivo não encontrado em {FILE_PATH}")

try:
    print(f"Iniciando processamento de {FILE_PATH}...")
    
    # Ingestão
    df = pd.read_csv(
        FILE_PATH, 
        usecols=COLS_MAP.keys(),
        encoding='latin1', # Verra geralmente é latin1
        low_memory=False
    )
    
    # Renomeação
    df = df.rename(columns=COLS_MAP)
    
    # 3. Limpeza Pesada (Data Cleaning)
    
    # A. Tratamento de Quantidade (Remover vírgulas e converter para número)
    # Ex: "1,500" (texto) -> 1500.0 (float)
    if df['quantity'].dtype == 'O': 
        df['quantity'] = df['quantity'].astype(str).str.replace(',', '').astype(float)
        
    # B. Tratamento de Datas
    # 'Vintage End' vem como data completa (2020-12-31). Extraímos só o ano (2020).
    df['vintage'] = pd.to_datetime(df['vintage'], errors='coerce').dt.year.fillna(0).astype(int)
    
    # 'Retirement Date' é crucial para saber se o crédito já foi consumido.
    df['retirement_date'] = pd.to_datetime(df['retirement_date'], errors='coerce')
    
    # C. Tratamento de Strings
    # Removemos espaços extras e padronizamos para Title Case
    df['country'] = df['country'].astype(str).str.strip().str.title()
    df['project_type'] = df['project_type'].astype(str).str.strip()

    print("-" * 30)
    print("ETAPA 1 E 2 CONCLUÍDAS: DADOS LIMPOS")
    print("-" * 30)
    print(f"Volume Total de Créditos Carregados: {df['quantity'].sum():,.0f}")
    df.info()
    display(df.head())

except Exception as e:
    print(f"[ERRO FATAL] O script falhou novamente. Detalhe: {e}")

Iniciando processamento de ./data/raw_data.csv...
------------------------------
ETAPA 1 E 2 CONCLUÍDAS: DADOS LIMPOS
------------------------------
Volume Total de Créditos Carregados: 1,366,661,902
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313803 entries, 0 to 313802
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   vintage          313803 non-null  int64         
 1   project_name     313803 non-null  object        
 2   country          313803 non-null  object        
 3   project_type     313803 non-null  object        
 4   quantity         313803 non-null  float64       
 5   retirement_date  292401 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 14.4+ MB


,vintage,project_name,country,project_type,quantity,retirement_date
0,2022,Huizhou Landfill Gas Power Generation Phase II...,China,Energy industries (renewable/non-renewable sou...,144100.0,NaT
1,2022,Grassland Restoration and Stewardship in South...,South Africa,Agriculture Forestry and Other Land Use,97440.0,NaT
2,2021,Grassland Restoration and Stewardship in South...,South Africa,Agriculture Forestry and Other Land Use,4750.0,NaT
3,2023,Grassland Restoration and Stewardship in South...,South Africa,Agriculture Forestry and Other Land Use,164064.0,NaT
4,2020,Madushan Hydropower Project on Honghe River in...,China,Energy industries (renewable/non-renewable sou...,40561.0,2026-01-17


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- CONFIGURAÇÃO INICIAL ---
sns.set_theme(style="whitegrid")

# ATENÇÃO: Verifique se os nomes dos arquivos na pasta batem com estes abaixo
# Adicionei o '.csv' pois o Python precisa da extensão completa
PATH_ISSUANCES = './data/raw_data_issuances.csv'   
PATH_PROJECTS  = './data/raw_data_pipeline.csv'  

print(">>> INICIANDO ETL (Extract, Transform, Load)...")

# 1. Carregar Emissões (O "Dinheiro")
print(f"1. Carregando: {PATH_ISSUANCES}...")
cols_issuance = {
    'ID': 'project_id', 
    'Name': 'project_name',
    'Country/Area': 'country',
    'Project Type': 'project_type',
    'Vintage End': 'vintage',
    'Quantity Issued': 'quantity',
}

try:
    df = pd.read_csv(PATH_ISSUANCES, usecols=cols_issuance.keys(), encoding='latin1', low_memory=False)
    df = df.rename(columns=cols_issuance)
except FileNotFoundError:
    print(f"❌ ERRO CRÍTICO: Não achei o arquivo '{PATH_ISSUANCES}'. Verifique o nome na pasta.")

# 2. Carregar Projetos (O "Dono")
print(f"2. Carregando: {PATH_PROJECTS}...")
cols_project = {
    'ID': 'project_id', 
    'Proponent': 'developer_company' # A coluna que queremos!
}

try:
    df_proj = pd.read_csv(PATH_PROJECTS, usecols=cols_project.keys(), encoding='latin1', low_memory=False)
    df_proj = df_proj.rename(columns=cols_project)
    
    # Remover duplicatas (um projeto pode ter várias linhas no pipeline, mas o dono é um só)
    df_proj = df_proj.drop_duplicates(subset=['project_id'])

    # 3. O MERGE (Cruzamento)
    print("3. Cruzando tabelas (Identificando as empresas)...")
    # Trazemos a coluna 'developer_company' para dentro da tabela de emissões
    df_merged = df.merge(df_proj, on='project_id', how='left')
    
    # Tratamento para projetos sem match
    df_merged['developer_company'] = df_merged['developer_company'].fillna('Não Identificado')
    df = df_merged
    
    n_match = df[df['developer_company'] != 'Não Identificado'].shape[0]
    print(f"   > Sucesso! {n_match:,} emissões foram vinculadas a uma empresa.")

except FileNotFoundError:
    print(f"⚠️ AVISO: Não achei '{PATH_PROJECTS}'. O código vai rodar, mas sem nomes de empresas.")
    df['developer_company'] = "Desconhecido"

# 4. Limpeza Final (Data Cleaning)
if df['quantity'].dtype == 'O': 
    df['quantity'] = df['quantity'].astype(str).str.replace(',', '').astype(float)

df['vintage'] = pd.to_datetime(df['vintage'], errors='coerce').dt.year.fillna(0).astype(int)
df['country'] = df['country'].astype(str).str.strip().str.title()

print("-" * 30)
print("BASE DE DADOS PRONTA.")
display(df.head(3))

>>> INICIANDO ETL (Extract, Transform, Load)...
1. Carregando: ./data/raw_data_issuances.csv...
2. Carregando: ./data/raw_data_pipeline.csv...
3. Cruzando tabelas (Identificando as empresas)...
   > Sucesso! 313,803 emissões foram vinculadas a uma empresa.
------------------------------
BASE DE DADOS PRONTA.


,vintage,project_id,project_name,country,project_type,quantity,developer_company
0,2022,3348,Huizhou Landfill Gas Power Generation Phase II...,China,Energy industries (renewable/non-renewable sou...,144100.0,"Shenzhen PhasCon Technologies Co., Ltd."
1,2022,2931,Grassland Restoration and Stewardship in South...,South Africa,Agriculture Forestry and Other Land Use,97440.0,Multiple Proponents
2,2021,2931,Grassland Restoration and Stewardship in South...,South Africa,Agriculture Forestry and Other Land Use,4750.0,Multiple Proponents
